In [ ]:
Data processing

In [ ]:
- outliers
	- analyse distributions of variables, remove outliers usin plots, distances
- compress data:
	- 
- transform variables
	- multivariante predicion variables! reduce to one? predict separately? predict all?
	- dolar based explanatory variables: is there a way to adjust for purchasing power?

In [ ]:
- analyse distributions of variables, remove outliers usin plots, distances

In [ ]:
data in time-series, reduce thw whole series to one observation? since the observations spawn only from 2000 - 2016, threre are no big, time-based insights hidden there!

In [ ]:
- multivariante predicion variables! reduce to one? predict separately? predict all?
- dolar based explanatory variables: is there a way to adjust for purchasing power?